In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_unixtime, unix_timestamp, from_json,regexp_replace, split
from pyspark.sql.functions import expr, concat, lit
from pathlib import Path

sc = SparkContext('local')
spark = SparkSession(sc)

# get passcode to pass into aes_encyrpt function to encyrpt credit card number
file = Path("/tmp/passcode.txt")
passcode = file.read_text()

# credit card data raw schema
credit_card_schema = StructType([
  StructField("Unnamed: 0", StringType()),
  StructField("trans_date_trans_time", StringType()),
  StructField("cc_bic", StringType()),
  StructField("cc_num", StringType()),
  StructField("merchant", StringType()),
  StructField("category", StringType()),
  StructField("amt", StringType()),
  StructField("personal_detail", StringType()), 
  StructField("trans_num", StringType()), 
  StructField("merch_lat", StringType()),
  StructField("merch_long", StringType()),
  StructField("is_fraud", StringType()),
  StructField("merch_zipcode", StringType()),
  StructField("merch_eff_time", StringType()),
  StructField("merch_last_update_time", StringType())
])

# read json file and print schema
df_with_schema = spark.read.schema(credit_card_schema) \
        .json("/home/jovyan/.cache/kagglehub/datasets/jinquan/cc-sample-data/versions/1")
df_with_schema.printSchema()

# schema for reference table uszip codes 
uszip_schema = StructType([
  StructField("zip", StringType()),
  StructField("lat", TimestampType()),
  StructField("lng", StringType()),
  StructField("city", StringType()),
  StructField("state_id", StringType()),
  StructField("state_name", StringType()),
  StructField("zcta", StringType()),
  StructField("parent_zcta", StringType()),
  StructField("population", StringType()),
  StructField("density", StringType()),
  StructField("county_fips", StringType()), 
  StructField("county_name", StringType()), 
  StructField("county_weights", StringType()),
  StructField("county_names_all", StringType()),
  StructField("county_fips_all", StringType()),
  StructField("imprecise", StringType()),
  StructField("military", StringType()),
  StructField("timezone", StringType())
])
df_uszip = spark.read.option("header","true").schema(uszip_schema).csv("/tmp/uszips.csv")


# schema for personal details for the inner json
personal_detail_schema = StructType(
    [
        StructField('person_name', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('address', StringType(), True),
        StructField('lat', StringType(), True),
        StructField('long', StringType(), True),
        StructField('city_pop', StringType(), True),
        StructField('job', StringType(), True),
        StructField('dob', StringType(), True)
    ]
)

# schema for address for the inner json
address_schema = StructType(
    [
        StructField('street', StringType(), True),
        StructField('city', StringType(), True),
        StructField('state', StringType(), True),
        StructField('zip', StringType(), True)
    ]
)

# transformation for credit card data
df = df_with_schema.withColumn("personal_detail", from_json("personal_detail", personal_detail_schema))\
    .withColumn("personal_detail_address", from_json("personal_detail.address", address_schema))\
    .withColumn("encrypted_cc_num", expr( "base64(aes_encrypt(cc_num, '"+str(passcode)+"', 'ECB', 'PKCS'))" ))\
    .select(col('Unnamed: 0').alias('id'),\
            col('encrypted_cc_num').alias('Credit Card Number'),\
            regexp_replace(col('merchant'),"fraud_","").alias('Merchant Name'),\
            col('category').alias('Merchant Category'),\
            col('trans_date_trans_time').cast(TimestampType()).alias('Transaction Time'),\
            col('amt').cast(DoubleType()).alias('Transaction Amount'),\
            regexp_replace(regexp_replace(col('personal_detail.person_name'), ",|@|\\||!|/|\\."," "),"\\s+"," ").alias('Person Name'),\
            col('personal_detail.gender').alias('Credit Card Owner\'s Gender'),\
            col('personal_detail_address.street').alias('personal_detail_address_street'),\
            col('personal_detail_address.city').alias('Credit Card Owner\'s City'),\
            col('personal_detail_address.state').alias('Credit Card Owner\'s State'),\
            col('personal_detail_address.zip').alias('Credit Card Owner\'s Zip Code'),\
            col('personal_detail.lat').cast(DecimalType()).alias('Credit Card Owner\'s Latitude'),\
            col('personal_detail.long').cast(DecimalType()).alias('Credit Card Owner\'s Longtitude'),\
            col('personal_detail.city_pop').cast(IntegerType()).alias('City Population'),\
            col('personal_detail.job').alias('Credit Card Owner\'s Job'),\
            col('personal_detail.dob').alias('personal_detail_dob'),\
            col('trans_num').alias('Transaction Number'),\
            col('merch_lat').cast(DecimalType()).alias('Merchant Latitude'),\
            col('merch_long').cast(DecimalType()).alias('Merchant Longtitude'),\
            col('is_fraud').alias('Fraud Case Indicator'),\
            col('merch_zipcode'),\
            from_unixtime(col("merch_last_update_time")/1000,"MM-dd-yyyy HH:mm:ss").alias("Merchant Last Update Time"),\
            from_unixtime(col("merch_eff_time")/1000000,"MM-dd-yyyy HH:mm:ss").alias("Merchant Effective Registration Time"),\
            col('cc_bic').alias('Credit Card BIC Code')).\
            withColumn("First Name", split(col("Person Name"), " ").getItem(0)).\
            withColumn("Last Name", split(col("Person Name"), " ").getItem(1))

# left join on merchant zipcode with reference table us zip code
result = df.join(df_uszip, df.merch_zipcode==df_uszip.zip, 'left')

# final result data frames
result = result.withColumn("Credit Card Owner\'s Street Address", expr( "base64(aes_encrypt(personal_detail_address_street, '"+str(passcode)+"', 'ECB', 'PKCS'))" ))\
        .withColumn("Credit Card Owner\'s Date of Birth", expr( "base64(aes_encrypt(personal_detail_dob, '"+str(passcode)+"', 'ECB', 'PKCS'))" ))\
        .select(col('id'),\
            col('Transaction Time'),\
            col('Credit Card Number'),\
            col('Merchant Name'),\
            col('Merchant Category'),\
            col('Transaction Amount'),\
            col('First Name'),\
            col('Last Name'),\
            col('Credit Card Owner\'s Gender'),\
            col('Credit Card Owner\'s Street Address'),\
            col('Credit Card Owner\'s City'),\
            col('Credit Card Owner\'s State'),\
            col('Credit Card Owner\'s Zip Code'),\
            col('Credit Card Owner\'s Latitude'),\
            col('Credit Card Owner\'s Longtitude'),\
            col('City Population'),\
            col('Credit Card Owner\'s Job'),\
            col('Credit Card Owner\'s Date of Birth'),\
            col('Transaction Number'),\
            col('Merchant Latitude'),\
            col('Merchant Longtitude'),\
            col('Fraud Case Indicator'),\
            col('merch_zipcode').alias('Merchant Zip Code'),\
            col("Merchant Last Update Time"),\
            col("Merchant Effective Registration Time"),\
            col('Credit Card BIC Code'),\
            concat(lit('US-'),col('state_id')).alias('Merchant State ID'),\
            col('state_name').alias('Merchant State Name'))

# write final result to csv
result.write.csv('/tmp/pyspark_csv',header='true')
